In [1]:
import pandas as pd

In [2]:
# Veriyi oku
df = pd.read_csv("superstore.csv", encoding='ISO-8859-1')

In [4]:
# EDA

def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    

check_df(df)

##################### Shape #####################
(9994, 21)
##################### Types #####################
Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object
##################### Head #####################
   Row ID        Order ID Order Date   Ship Date     Ship Mode Customer ID  \
0       1  CA-2016-152156  11/8/2016  11/11/2016  Second Class    CG-12520   
1       2  CA-2016-152156  11/8/2016  11/11/2016  Second Class    CG-12520   
2       3  CA-2016-138688  6/12/2016   6/16/2016  Second Class    D

In [5]:
# Tip dönüşümleri 
df['Order Date'] = pd.to_datetime(df['Order Date'], format="%m/%d/%Y")
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format="%m/%d/%Y")

In [6]:
# Önemli kolonlar türet
df['Shipping Duration days'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Cost'] = df['Sales'] - df['Profit']


In [7]:
# Gereksiz kolonları sil
df.drop(columns=['Row ID', 'Country'], inplace=True)

In [8]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [8]:
# Boyut (Dimension) Tabloları


# dim_customer
dim_customer = df[['customer_id', 'customer_name', 'segment', 'city', 'state', 'region']].drop_duplicates().reset_index(drop=True)

# dim_product
dim_product = df[['product_id', 'product_name', 'category', 'sub-category']].drop_duplicates().reset_index(drop=True)

# dim_date
df['order_date'] = pd.to_datetime(df['order_date'])  # Tarih dönüşümü (gerekirse)
dim_date = df[['order_date']].drop_duplicates().reset_index(drop=True)
dim_date['order_date_id'] = dim_date['order_date'].dt.strftime('%Y%m%d').astype(int)
dim_date['order_day'] = dim_date['order_date'].dt.day
dim_date['order_month'] = dim_date['order_date'].dt.month
dim_date['order_year'] = dim_date['order_date'].dt.year
dim_date['order_weekday'] = dim_date['order_date'].dt.day_name()


# Fakt (Fact) Tablosu


fact_sales = df[[
    'order_id', 'order_date', 'product_id', 'customer_id',
    'sales', 'quantity', 'discount', 'profit', 'cost', 'shipping_duration_days'
]].copy()

fact_sales['order_date_id'] = fact_sales['order_date'].dt.strftime('%Y%m%d').astype(int)
fact_sales.drop(columns=['order_date'], inplace=True)



In [9]:
# Dim tablolarındaki tekrarlı kayıtları silme ve index sıfırlama
dim_customer = dim_customer.drop_duplicates(subset='customer_id').reset_index(drop=True)
dim_product = dim_product.drop_duplicates(subset='product_id').reset_index(drop=True)
dim_date = dim_date.drop_duplicates(subset='order_date_id').reset_index(drop=True)

In [10]:
# CSV olarak kaydet
dim_customer.to_csv("/home/train/datasets/dim_customer.csv", index=False)
dim_product.to_csv("/home/train/datasets/dim_product.csv", index=False)
dim_date.to_csv("/home/train/datasets/dim_date.csv", index=False)
fact_sales.to_csv("/home/train/datasets/fact_sales.csv", index=False)
